<a href="https://colab.research.google.com/github/MarcelloCeresini/ChessBreaker/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
username = 'MarcelloCeresini'
repository = 'ChessBreaker'

In [2]:
# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !nvidia-smi             # Check which GPU has been chosen for us
    !rm -rf logs
    #from google.colab import drive
    #drive.mount('/content/drive')
    #%cd /content/drive/MyDrive/GitHub/
    !git clone https://github.com/{username}/{repository}.git
    %cd {repository}
    %ls
    !pip3 install anytree
except:
    IN_COLAB = False

In [5]:
import numpy as np
import tensorflow as tf
import chess
import utils

from utils import plane_dict, Config, x_y_from_position

conf = Config()
board = chess.Board()

legal_moves = board.legal_moves
for move in legal_moves:
    print(move.uci())  
print(legal_moves)


g1h3
g1f3
b1c3
b1a3
h2h3
g2g3
f2f3
e2e3
d2d3
c2c3
b2b3
a2a3
h2h4
g2g4
f2f4
e2e4
d2d4
c2c4
b2b4
a2a4
<LegalMoveGenerator at 0x7f5c880ff7c0 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>


In [6]:
# print(board.fen())
print(list(board.pieces(1,True)))

[8, 9, 10, 11, 12, 13, 14, 15]


In [72]:
def uniform_tensor(x):
    return tf.fill(conf.BOARD_SHAPE, x)

INITIAL_NORMAL_PLANES = tf.zeros([*conf.BOARD_SHAPE, conf.PAST_TIMESTEPS*conf.REPEATED_PLANES], dtype=conf.PLANES_DTYPE)

def special_input_planes(board):
    return tf.transpose(tf.vectorized_map( 
            uniform_tensor,
            tf.constant([
                int(board.turn                                 ),
                int(board.fullmove_number-1                    ),   # don't know why but it starts from 1 on move 1, just reduce it by one and now it's right  
                int(board.has_kingside_castling_rights(True)   ),   # True for White
                int(board.has_queenside_castling_rights(True)  ),
                int(board.has_kingside_castling_rights(False)  ),   # False for Black
                int(board.has_queenside_castling_rights(False) ),
                int(board.halfmove_clock                       )
            ], dtype=conf.PLANES_DTYPE)
        ), [1,2,0])

# def update_normal_planes(current, board, board_history):
def update_normal_planes(current, board):
    planes = [] # since we cannot "change" a tensor after creating it, we create them one by one in a list and then stack them

    for color in range(2):                                                  # for each color
        for piece_type in range(1, conf.N_PIECE_TYPES+1):                   # for each piece type
            indices = []                                                    # we save the position on the board in a list
            for position in list(board.pieces(piece_type, color)):          # for each piece of that type
                indices.append(x_y_from_position(position))                 # the function transforms a number (1-64) into a tuple (1-8, 1-8)
            values = np.array([1]*len(indices), dtype=conf.PLANES_DTYPE_NP) # simply "1" in a list with unit8 dtype
            tensor = tf.sparse.to_dense(tf.SparseTensor(dense_shape=[*conf.BOARD_SHAPE], indices=indices, values=values)) ### created as sparse because it's easier, needed as dense afterwards
            planes.append(tensor)
        planes.append(tf.zeros([*conf.BOARD_SHAPE], dtype=conf.PLANES_DTYPE))   # adding a "repetition plane" for each color # TODO: actually add a repetition counter through "board_history"

    current_planes = tf.transpose(tf.stack(planes), [1,2,0])                # transpose them to have the planes as last dimension
    old_planes = tf.slice(current, begin=[0,0,0], size=[*conf.BOARD_SHAPE, (conf.PAST_TIMESTEPS-1)*conf.REPEATED_PLANES]) # take the first 7 repetitions, slice them and paste them at the end of the new planes
    
    return tf.concat([current_planes, old_planes], axis=-1)


def full_input_planes(board, current_normal_planes=None):
    if current_normal_planes == None:
        current_normal_planes = INITIAL_NORMAL_PLANES
    new_normal = update_normal_planes(current_normal_planes, board)
    new_total = tf.concat([
                    new_normal, 
                    special_input_planes(board)
                ], axis = -1)
    return new_total, new_normal

In [ ]:
from anytree import Node

def choose_move(board, root_node):
    children = root_node.children
    if root_node.children != []:
        root = np.argmax( [child.value for child in children] ) # choose the move with the highest value to be played
        root.parent = None # To detach the subtree and restart with the next move search

    return board.push(root.name), root

def MTCS(board, root_node):
    legal_moves = board.legal_moves
    if root_node.is_leaf:
        priors = tf.boolean_mask(model(full_input_planes(board)), utils.mask_moves(list(legal_moves)))
        for move, prior in zip(legal_moves, priors):
            Node(move, parent = root_node, prior = prior)
    for child in 

def complete_search(board):
    root_node = Node("", value=0)
    while not board.is_game_over(claim_draw=True):
        MTCS(board, root_node)
        (new_board, new_root_node) = choose_move(board, root_node)
        board, root_node = new_board, new_root_node

        for move in board.legal_moves:

            Node(move, parent = root)


In [ ]:
help(Node)

Help on class Node in module anytree.node.node:

class Node(anytree.node.nodemixin.NodeMixin, builtins.object)
 |  Node(name, parent=None, children=None, **kwargs)
 |  
 |  Method resolution order:
 |      Node
 |      anytree.node.nodemixin.NodeMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, parent=None, children=None, **kwargs)
 |      A simple tree node with a `name` and any `kwargs`.
 |      
 |      Args:
 |          name: A name or any other object this node can reference to as idendifier.
 |      
 |      Keyword Args:
 |          parent: Reference to parent node.
 |          children: Iterable with child nodes.
 |          *: Any other given attribute is just stored as object attribute.
 |      
 |      Other than :any:`AnyNode` this class has at least the `name` attribute,
 |      to distinguish between different instances.
 |      
 |      The `parent` attribute refers the parent node:
 |      
 |      >>> from anytree import Node, R

In [ ]:
def training_loop():
    